In [1]:
# !pip install tensorflow==2.1.0

In [2]:
import tensorflow as tf 
print(tf.__version__)

2.1.0


In [3]:
# !pip install -q tensorflow-datasets 

In [4]:
import tensorflow_datasets as tfds 
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.2)
config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)

imdb,info=tfds.load('imdb_reviews/subwords8k',with_info=True,as_supervised=True) 

In [5]:
train_data1,test_data1=imdb['train'],imdb['test'] 

In [6]:
tokenizer=info.features['text'].encoder 
 

In [7]:
sample_string='TensorFlow, from basic to mastery'
tokenized_string =tokenizer.encode(sample_string) 
print('Tokenized string is {}'.format(tokenized_string))

Tokenized string is [6307, 2327, 4043, 2120, 2, 48, 2715, 7, 2652, 8050]


In [8]:
original_string=tokenizer.decode(tokenized_string) 
print('The original string : {}'.format(original_string))

The original string : TensorFlow, from basic to mastery


In [9]:
for ts in tokenized_string:
    print('{} ----->{}'.format(ts,tokenizer.decode([ts])))

6307 ----->Ten
2327 ----->sor
4043 ----->Fl
2120 ----->ow
2 ----->, 
48 ----->from 
2715 ----->basic 
7 ----->to 
2652 ----->master
8050 ----->y


In [11]:
embedding_dim = 64
max_length = 120

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
#     tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary() 

In [ ]:
tokenizer.vocab_size*embedding_dim

In [ ]:
num_epochs = 50

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# train_data = train_data1.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)),num_parallel_calls=4)
# test_data = test_data1.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)),num_parallel_calls=4)

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_data1 = train_data1.shuffle(BUFFER_SIZE)
train_data = train_data1.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_data1))
test_data= test_data1.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_data1))

history = model.fit(train_data, epochs=num_epochs, validation_data=test_data)

In [ ]:
import matplotlib.pyplot as plt 
def plot_graphs(history,string):
    plt.plot(history.history[string]) 
    plt.plot(history.history['val_'+string]) 
    plt.xlabel('Epochs') 
    plt.ylabel(string) 
    plt.legend([string,'val_'+string]) 
    plt.show() 

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
train_data

In [ ]:
for a in train_data:
    print(a) 
    break 